In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
plt.style.use("ggplot")

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier

# Preprocessing

In [3]:
data_raw = pd.read_csv("monthly.csv")
y_raw = pd.read_csv("y.csv")

In [4]:
y_raw.head()

,date,TICKER,Y
0,19650129,NCM,1
1,19650129,ACD,0
2,19650129,ABP,1
3,19650129,T,1
4,19650129,AME,1


In [5]:
data_raw.head()

,PERMNO,date,SHRCD,TICKER,COMNAM,PRC,VOL,RET,RETX
0,10057,19650129,11.0,NCM,NATIONAL ACME CO,71.75,110.0,0.039855,0.039855
1,10057,19650226,11.0,NCM,NATIONAL ACME CO,71.25,82.0,0.001742,-0.006969
2,10057,19650331,11.0,NCM,NATIONAL ACME CO,76.00,243.0,0.066667,0.066667
3,10057,19650430,11.0,NCM,NATIONAL ACME CO,39.50,152.0,0.039474,0.039474
4,10057,19650528,11.0,NCM,NATIONAL ACME CO,37.00,256.0,-0.054430,-0.063291


In [6]:
if not isinstance(data_raw['date'][0], str):
    data_raw["date"] = data_raw["date"].apply(lambda x: str(x)[:4]+'-'+str(x)[4:6]+'-'+str(x)[6:])
data_raw["dateDT"] = pd.to_datetime(data_raw["date"])

if not isinstance(y_raw['date'][0], str):
    y_raw["date"] = y_raw["date"].apply(lambda x: str(x)[:4]+'-'+str(x)[4:6]+'-'+str(x)[6:])
y_raw["dateDT"] = pd.to_datetime(y_raw["date"])

In [7]:
data_raw = data_raw.sort_values(by=["dateDT", "TICKER"])
data_raw.index = range(data_raw.shape[0])

In [8]:
y_raw = y_raw.sort_values(by=["dateDT", "TICKER"])
y_raw.index = range(y_raw.shape[0])

In [9]:
data_raw["id"] = data_raw.index

In [10]:
# split train, val, test
train_val_split = int(data_raw.shape[0]*0.4)
val_test_split = int(data_raw.shape[0]*0.6)
data_train = data_raw.loc[data_raw["dateDT"] <= data_raw["dateDT"][train_val_split]]
train_cut = data_train.shape[0]
data_val = data_raw.iloc[train_cut:].loc[data_raw["dateDT"] <= data_raw["dateDT"][val_test_split]]
val_cut = train_cut + data_val.shape[0]
data_test = data_raw.iloc[val_cut:]

In [11]:
y_train = y_raw.iloc[:train_cut]
y_val = y_raw.iloc[train_cut:val_cut]
y_test = y_raw.iloc[val_cut:]

In [12]:
feature_all = ['PRC', 'VOL', 'RET']

# Descriptive Analysis

# Basic Models